In [8]:
import torch as th
import numpy as np
import pickle
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from pytorch_lightning import seed_everything
import torch as th
import tqdm
import os, glob

set_ = 'train'
img_file = glob.glob(f"/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/{set_}/*.jpg")
img_size = 256
import cv2
from PIL import Image
import blobfile as bf

def sobel_edge(image, device='cuda'):
    # image is in [B, 1, H, W] format
    assert image.shape[1] == 1
    
    sobel_x = th.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=th.float32).to(device)
    sobel_y = th.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=th.float32).to(device)

    # Apply Sobel operators to compute gradients
    gradient_x = F.conv2d(image, sobel_x.view(1, 1, 3, 3), padding='same')
    gradient_y = F.conv2d(image, sobel_y.view(1, 1, 3, 3), padding='same')

    # Compute the magnitude of the gradient
    gradient_magnitude = th.sqrt((gradient_x**2 + gradient_y**2) + 1e-6)
    # gradient_magnitude = th.sqrt(th.clamp((gradient_x**2 + gradient_y**2), 0.0) + 1e-6)

    return gradient_x, gradient_y, gradient_magnitude

mode = 'L'
for i, path in enumerate(tqdm.tqdm(img_file)):
    name = path.split('/')[-1].split('.')[0]
    with bf.BlobFile(path, "rb") as f:
        pil_image = Image.open(f)
        pil_image.load()
    pil_image = pil_image.convert(mode)
    print("Max-Min of load image: ", np.max(np.array(pil_image)), np.min(np.array(pil_image)))
    print("Shape: ", np.array(pil_image).shape)
    img = np.array(pil_image) / 255
    img = img[None, ..., None]
    img = th.from_numpy(img).permute(0, 3, 1, 2).float().cuda()
    gx, gy, gmag = sobel_edge(img)
    print("Max-Min of sobel image: ", th.max(gmag), th.min(gmag))
    print("Max-Min of sobel-x image: ", th.max(gx), th.min(gx))
    print("Max-Min of sobel-y image: ", th.max(gy), th.min(gy))
    # Visualize all gradients
    plt.figure(figsize=(15, 15))
    plt.imshow(th.cat((img[0], gx[0], gy[0], gmag[0]), dim=2).cpu().permute(1, 2, 0).repeat(1, 1, 3))
    plt.show()
    if i == 3: break
    

  0%|          | 0/60000 [00:00<?, ?it/s]

Max-Min of load image:  254 5
Shape:  (256, 256)
Max-Min of sobel image:  tensor(3.2999, device='cuda:0') tensor(0.0010, device='cuda:0')
Max-Min of sobel-x image:  tensor(2.8000, device='cuda:0') tensor(-3.2745, device='cuda:0')
Max-Min of sobel-y image:  tensor(2.3843, device='cuda:0') tensor(-3.1451, device='cuda:0')


ValueError: Unsupported dtype

<Figure size 1080x1080 with 1 Axes>

Max-Min of load image:  252 29
Shape:  (256, 256)
Max-Min of sobel image:  tensor(3.7216, device='cuda:0') tensor(0.0010, device='cuda:0')
Max-Min of sobel-x image:  tensor(2.5569, device='cuda:0') tensor(-3.7216, device='cuda:0')
Max-Min of sobel-y image:  tensor(2.1490, device='cuda:0') tensor(-2.7961, device='cuda:0')


...

  0%|          | 3/60000 [00:00<40:14, 24.85it/s]

Max-Min of load image:  223 1
Shape:  (256, 256)
Max-Min of sobel image:  tensor(2.4456, device='cuda:0') tensor(0.0010, device='cuda:0')
Max-Min of sobel-x image:  tensor(1.6980, device='cuda:0') tensor(-2.2824, device='cuda:0')
Max-Min of sobel-y image:  tensor(2.0275, device='cuda:0') tensor(-2.0392, device='cuda:0')


ValueError: Unsupported dtype

<Figure size 1080x1080 with 1 Axes>

  0%|          | 3/60000 [00:00<53:21, 18.74it/s]
